Sree Ganeshaaya Namaha

# 1. Read Events from a Big File using pySpark
# 2. Remove the unwanted events using Spark DataFrame APIs
# 3. Add new column to represent EPOCH time
# 4. ==> Following doesn't work now, proabably, as I have used spark 2.3 with java 11, So use Spark itself to write the out file 
         # (Convert filtered list to a pandas dataframe => Spark dataframe.write.csv write operation writes segmented files. So, let us use pandas.)
         # 5. Write the padas dataframe to the .csv file=> This will be a smaller file in comparison to the raw file and will be used for further processing.



# Import Libraries

In [1]:
import findspark
findspark.init("C:\myprograms\spark-2.3.0-bin-hadoop2.7")

import pyspark
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#udf, concat, desc, window

from pyspark.sql.types import *
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
# Use tqdm to show progress of an pandas function we use
tqdm.pandas()


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

# Create SparkSql Session
use local node with 2 CPUs as master. 

In [2]:
spark=SparkSession.builder.appName(name="filter_file").master("local[2]").getOrCreate()


In [3]:
df=spark.read.csv("input/raw_events.csv", header=True)

In [19]:
df.columns
#following gives error due to java 11 - df.select('HOST').count()


['TIMESTAMP',
 'HOST',
 'FACILITY',
 'SEVERITY',
 'EPOLICY',
 'EMESSAGE',
 'SUPPRESSION_FLAG']

# SELECT only necessary columns

In [20]:
df2=df.select("TIMESTAMP","HOST","EMESSAGE")
df2.columns


['TIMESTAMP', 'HOST', 'EMESSAGE']

In [ ]:
#following code is supposed to work, but it doesn't work.
#Let us give some time for pyspark to improve. 
#we use the other alternative to filter for now.
#for more details - https://stackoverflow.com/questions/51710126/pyspark-regex-to-data-frame/54859546#54859546
"""
t=spark.createDataFrame([(1,"hello a1"),(2,"hello b1")])
t.show()
def t_filter(tline):
    
    print(tline)
    if tline.contains("a1"):
        return False
    return True
t_udf=udf(t_filter,BooleanType())
t2=t.filter(t_udf(t._2))
t2.show()
"""

In [25]:
filter_arr=["authen_faile","sysmgr-","recvd_pkt_inv_spi","-power warning cleared","-authen_success",
            "sysmgr-standby-","mgbl-exec-","install-instmgr-","-system_msg","duplicate ip address","mgbl-tty-"
            "sysdb-sysdb-","starservicevpnname","ccmhistoryeventcommandsource","\%platform-",
            " threshold:"," - interface utilization","is active as part of bundle",
            "are removed from suspended state","counter overrun for one or more deltas",
            "changed state to up","security-tacacsd-%-server_down","security-tacacsd-%-server_up",
            "routing-ldp-%-rx_peer_dup_addr","ip-tcp-%-badauth","security-mpp-%-msg_info",
            "routing-ldp-%-peer_dup_addrs","bfd session to .%is up","mgbl-ifstats-%-collector_timeout","os-dumper-"]

Remove all rows where EMESSAGE column has one of the words given in filter_arr

In [26]:
for phrase in filter_arr:
    df2.createOrReplaceTempView("df_temp")
    df2 = spark.sql("select *, case when lower(EMESSAGE) like '%{}%' then 'TRUE' else 'FALSE' end filter_col from df_temp".format(phrase))
    df2=df2.filter("filter_col == FALSE").select("TIMESTAMP","HOST","EMESSAGE")
df2.show()


+-------------------+-------------+--------------------+
|          TIMESTAMP|         HOST|            EMESSAGE|
+-------------------+-------------+--------------------+
|02/20/2015 11:02:26|   10.17.3.18|LC/0/11/CPU0:Feb ...|
|02/20/2015 11:02:50|    10.17.3.4|LC/0/11/CPU0:Feb ...|
|02/20/2015 15:02:09|   10.17.3.18|LC/0/11/CPU0:Feb ...|
|02/20/2015 15:02:50|    10.17.3.4|LC/0/11/CPU0:Feb ...|
|02/20/2015 19:02:09|   10.17.3.18|LC/0/11/CPU0:Feb ...|
|09/29/2015 01:09:36|   10.17.3.82|LC/0/11/CPU0:Sep ...|
|09/29/2015 01:09:28|   10.17.3.18|LC/0/11/CPU0:Sep ...|
|09/29/2015 03:09:53|172.29.10.243|Sep 29 03:01:52.0...|
|09/29/2015 03:09:09|   10.17.3.82|LC/0/11/CPU0:Sep ...|
|09/29/2015 04:09:01|172.29.10.243|Sep 29 04:15:00.3...|
|09/29/2015 06:09:48|172.29.10.243|Sep 29 06:08:48.7...|
|09/29/2015 06:09:35|172.29.10.243|Sep 29 06:09:34.7...|
|09/29/2015 06:09:29|172.29.10.243|Sep 29 06:37:28.6...|
|09/29/2015 08:09:49| 10.17.131.27|LC/0/0/CPU0:Sep 2...|
|09/29/2015 08:09:49| 10.17.131

# toPandas.collect().to_csv() is not working yet in pyspark.
# Let us write using spark itself.

In [27]:
df2.repartition(1).write.csv("ouput/spark_filtered.csv", sep=',')
#df2.toPandas().collect().to_csv("faults/jaykumar/filtered_4.csv")


Following block is just for info

In [ ]:
#from pyspark.sql.functions import pandas_udf, PandasUDFType

# Use pandas_udf to define a Pandas UDF
#@pandas_udf('double', PandasUDFType.SCALAR)
# Input/output are both a pandas.Series of doubles

#def pandas_plus_one(v):
#    return v + 1
